<a href="https://colab.research.google.com/github/omar4646/My_repository/blob/main/growth_option_cal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
# Replace 'path/to/your/file.xlsx' with the actual path to your file in Google Drive
# For example, if your file is in the root of your Drive, the path would be '/content/drive/MyDrive/your_file_name.xlsx'
# If it's in a folder called 'data', the path would be '/content/drive/MyDrive/data/your_file_name.xlsx'
d = '/content/drive/MyDrive/dependent_var.xlsx'
w = '/content/drive/MyDrive/wacc_for_merger.xlsx'
f = '/content/drive/MyDrive/fcf_for_merger.xlsx'
wacc_for_merger = pd.read_excel(w)
dependent_var = pd.read_excel(d)
fcf_for_merger = pd.read_excel(f)


In [14]:
# Merge dependent_var and wacc_for_merger
merged_df = pd.merge(dependent_var, wacc_for_merger, on=['Symbol', 'EndDate'], how='inner')

# Merge the result with fcf_for_merger
final_merged_df = pd.merge(merged_df, fcf_for_merger, on=['Symbol', 'EndDate'], how='inner')

display(final_merged_df.head())

,CountryName_EN,Short_name,Symbol,EndDate,TotalAssets,perc_change,Unnamed: 6,number_of_firms,number_of_countries,MarketValue_cny,WACC_percent,F062401B
0,Afghanistan,JXTY,600362,2008,7.781770e+08,NaN,NaN,1236.0,105.0,3.016788e+10,6.3574,-2.418543e+09
1,Afghanistan,JXTY,600362,2009,1.044137e+09,0.293992,NaN,NaN,NaN,1.215481e+11,8.2132,-1.568481e+08
2,Afghanistan,JXTY,600362,2010,1.250250e+09,0.180153,NaN,NaN,NaN,1.564115e+11,8.5189,-5.728960e+09
3,Afghanistan,JXTY,600362,2011,1.860656e+09,0.397586,NaN,NaN,NaN,7.593766e+10,8.7417,4.318861e+09
4,Afghanistan,JXTY,600362,2012,2.304670e+09,0.214008,NaN,NaN,NaN,8.262072e+10,8.1772,8.154327e+08


In [16]:
# Drop specified columns
columns_to_drop = ['Unnamed: 6', 'number_of_firms', 'number_of_countries']
final_merged_df = final_merged_df.drop(columns=columns_to_drop, errors='ignore')

# Rename columns
new_column_names = {
    'Symbol': 'company_symbol',
    'EndDate': 'fiscal_year',
    'F062401B': 'free_cash_flow',
    'MarketValue_cny': 'market_value',
    'WACC_percent': 'wacc',
    'CountryName_EN': 'country_name',
    'Short_name': 'firm_name',
    'TotalAssets': 'foreign_asset',
    'perc_change': 'foreign_asset_change'
}
final_merged_df = final_merged_df.rename(columns=new_column_names)

display(final_merged_df.head())

,country_name,firm_name,company_symbol,fiscal_year,foreign_asset,foreign_asset_change,market_value,wacc,free_cash_flow
0,Afghanistan,JXTY,600362,2008,7.781770e+08,NaN,3.016788e+10,6.3574,-2.418543e+09
1,Afghanistan,JXTY,600362,2009,1.044137e+09,0.293992,1.215481e+11,8.2132,-1.568481e+08
2,Afghanistan,JXTY,600362,2010,1.250250e+09,0.180153,1.564115e+11,8.5189,-5.728960e+09
3,Afghanistan,JXTY,600362,2011,1.860656e+09,0.397586,7.593766e+10,8.7417,4.318861e+09
4,Afghanistan,JXTY,600362,2012,2.304670e+09,0.214008,8.262072e+10,8.1772,8.154327e+08


In [19]:
final_merged_df['wacc'] = final_merged_df['wacc'] / 100

display(final_merged_df.head())

,country_name,firm_name,company_symbol,fiscal_year,foreign_asset,foreign_asset_change,market_value,wacc,free_cash_flow
0,Afghanistan,JXTY,600362,2008,7.781770e+08,NaN,3.016788e+10,0.063574,-2.418543e+09
1,Afghanistan,JXTY,600362,2009,1.044137e+09,0.293992,1.215481e+11,0.082132,-1.568481e+08
2,Afghanistan,JXTY,600362,2010,1.250250e+09,0.180153,1.564115e+11,0.085189,-5.728960e+09
3,Afghanistan,JXTY,600362,2011,1.860656e+09,0.397586,7.593766e+10,0.087417,4.318861e+09
4,Afghanistan,JXTY,600362,2012,2.304670e+09,0.214008,8.262072e+10,0.081772,8.154327e+08


In [21]:
final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14950 entries, 0 to 14949
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country_name          14950 non-null  object 
 1   firm_name             14950 non-null  object 
 2   company_symbol        14950 non-null  int64  
 3   fiscal_year           14950 non-null  int64  
 4   foreign_asset         14950 non-null  float64
 5   foreign_asset_change  12485 non-null  float64
 6   market_value          14950 non-null  float64
 7   wacc                  14950 non-null  float64
 8   free_cash_flow        14950 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 1.0+ MB


In [22]:
final_merged_df['GO'] = 1 - (final_merged_df['free_cash_flow'] / (final_merged_df['market_value'] * final_merged_df['wacc']))


In [23]:
final_merged_df.head()

,country_name,firm_name,company_symbol,fiscal_year,foreign_asset,foreign_asset_change,market_value,wacc,free_cash_flow,GO
0,Afghanistan,JXTY,600362,2008,7.781770e+08,NaN,3.016788e+10,0.063574,-2.418543e+09,2.261042
1,Afghanistan,JXTY,600362,2009,1.044137e+09,0.293992,1.215481e+11,0.082132,-1.568481e+08,1.015712
2,Afghanistan,JXTY,600362,2010,1.250250e+09,0.180153,1.564115e+11,0.085189,-5.728960e+09,1.429956
3,Afghanistan,JXTY,600362,2011,1.860656e+09,0.397586,7.593766e+10,0.087417,4.318861e+09,0.349397
4,Afghanistan,JXTY,600362,2012,2.304670e+09,0.214008,8.262072e+10,0.081772,8.154327e+08,0.879304
